In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import splat
%matplotlib inline




Welcome to the Spex Prism Library Analysis Toolkit (SPLAT)!
If you make use of any features of this toolkit for your research, please remember to cite the SPLAT paper:

Burgasser et al. (2017, Astro. Soc. India Conf. Series 14, p. 7); Bibcode: 2017ASInC..14....7B

If you make use of any spectra or models in this toolkit, please remember to cite the original source.
Please report any errors are feature requests to our github page, https://github.com/aburgasser/splat/




In [2]:
splat.initializeStandards()

In [3]:
#short names for visually confirmed ugly spectra 
FORBIDDEN_LIST=['J0148+1202',  'J0331+4130', 'J0338-4409', 'J0343+3155',
' J0344+3204', '0344+3200', 'J0344+3156', 'J0344+3203',
'J0345+3205','J0419+2712', 'J0435+2254','J0438+2519',
'J0448-4432', 'J0448-4432', 'J0501-3341', 'J0512-2949',
'J0610-2151','J0621+6558','J0624-1045', 'J0628-0959',
'J0815+1041','J0822+1700', 'J0935+0019','J0950+0540',
'J1004+5023','J1050-2940','J1132-3018','J1132-3018',
'J1132-3018','J1132-3018','J1132-3018','J1132-3018',
'J1132-3018','J1132-3018', 'J1132-3018','J1132-3018',
'J1138-1314','J1209-3041','J1211-2821','J1224-2744',
'J1257-0204','J1303+2351','J1312+0051','J1317-1427',
'J1325-2128', 'J1420-1752', 'J1423+0116','J1629+1415',
'J1642-2355','J1642-2355','J1659+3515','J1726-1158',
'J1729+4352','J1829+5032','J1839-3744','J1924+5506',
'J1932+0133', 'J1932-3921', 'J1945-4149', 'J2001-3805',
'J2024-3422', 'J2028+6925', 'J2034+6727','J2151-3349',
'J2012+6726', 'J2043-1551', 'J2238+4353', 'J2302-0935',
'J0345+3205', 'J0412+1044', 'J0422+0810', 'J0439-2353',
'J0441+2130', 'J0729-3954', 'J0755+1138', 'J0919+5135',
'J1132-3018', 'J1423+0116', 'J1707-0538', 'J1726-1158',
'J1740+1317', 'J0248+2445', 'J0307+2904', 'J0330+3505',
'J0343+3203', 'J0343+3202','J0344+3208', 'J0344+3203',
'J0344+3201', 'J1620-2446', 'J0345+3205', 'J30345+3206', 
'J1659+2055','J0107-1314', 'J1039+3256', 'J1059+5659',
'J1202+5345', 'J1318+3626', 'J1619-2832', 'J1707-0558',
'J1813+5101', 'J2002-0521', 'J2300+0200']

In [4]:
def plot_sp_sp(sp, a, shortname, spex_type):
    #plot a spectrum
    std=splat.STDS_DWARF_SPEX[spex_type]
    
    chi, sc=splat.compareSpectra( sp, std)
    std.scale(sc)
    
    a.plot(sp.wave, sp.flux, c='k', linewidth=4, label=shortname)
    a.plot(std.wave, std.flux, c='#7FDBFF', linewidth=2, label='{} STD'.format(spex_type))
    
    xlim=[0.7, 2.5]
    mask= np.logical_and(std.wave.value>xlim[0], std.wave.value<xlim[1])
    ylim= [std.flux[mask].value.min(), std.flux.value[mask].max()]
    a.set(xlabel='Wave (micron)', ylabel=' Flux', xlim=xlim, ylim=ylim)
    
    a.legend()
    
def get_spectrum(filename):
    try:
        return splat.getSpectrum(filename=filename)[0]
    except:
        return 
    
def get_shortname(des):
    if '+' in des:
        c=des.split('+')
        return c[0][:5]+'+'+c[1][:4]
    if '-' in des:
        c=des.split('-')
        return c[0][:5]+'-'+c[1][:4]

In [5]:
%%capture
db= splat.searchLibrary(vlm=True)
spectra= db.DATA_FILE.apply(get_spectrum)

In [6]:
db['SHORTNAME']=db.DESIGNATION.apply(get_shortname)

In [7]:
#ADD A KEY WORK FOR OBJECTS THAT PASSED VISUAL INSPECTION
db['PASSED_INSPECTION']=[x not in FORBIDDEN_LIST for x in db.SHORTNAME ]

In [8]:
db['SPECTRA']= spectra

In [9]:
#db

In [10]:
db=(db[~db.SPEX_TYPE.isna()]).reset_index(drop=True)

In [11]:
#fig, ax=plt.subplots()
#plot_sp_sp(db.iloc[0].SPECTRA, ax, \
#           db.iloc[0].SHORTNAME, db.iloc[0].SPEX_TYPE)

In [12]:
from matplotlib.backends.backend_pdf import PdfPages

In [13]:
#'L0.0:'[:4]
#s[:4]

In [17]:
#plot everything in a pdf
def plot_everything(df, filename):
    with PdfPages('/users/caganze/'+filename) as pdf:

        for g in np.array_split(df, int(len(df)/4)):
            

            fig, ax=plt.subplots(ncols=2, nrows=2)
            
            plot_sp_sp(g.SPECTRA.iloc[0], ax[0][0], g.SHORTNAME.iloc[0],  g.SPEX_TYPE.iloc[0][:4] )
            plot_sp_sp(g.SPECTRA.iloc[1], ax[0][1],  g.SHORTNAME.iloc[1], g.SPEX_TYPE.iloc[1][:4] )
            plot_sp_sp(g.SPECTRA.iloc[2], ax[1][0],  g.SHORTNAME.iloc[2], g.SPEX_TYPE.iloc[2][:4] )
            plot_sp_sp(g.SPECTRA.iloc[3], ax[1][1],  g.SHORTNAME.iloc[3], g.SPEX_TYPE.iloc[3][:4] )
            
            plt.tight_layout()
            pdf.savefig() 
            plt.close()
            


In [15]:
plot_everything(db[db.PASSED_INSPECTION].reset_index(drop=True), filename='vlm_splat_spectra.pdf')
plot_everything(db[~db.PASSED_INSPECTION].reset_index(drop=True), filename='rejected_spectra.pdf')

In [16]:
#scale to the appropriate flux using Dupuy relations
#interpolate on wavegrid (0.9-2.4 micron),
#compute photometric spectro-photometric magnitudes in MKO J and H
#measure J-SNR 1.1-1.3 micron
#